In [ ]:
import os
import sys
import time
import numpy as np
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Box

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from Agents.agent import Agent
from Agents.RayCast import get_fov_rays
from Constants.constants import WHITE, RED, BLUE, SCREEN_WIDTH, SCREEN_HEIGHT, WALLS, WALLS2
from Walls.collision_detection import detect_collision
from Walls.wall_class import Walls




In [ ]:

class GameEnv(Env):
    def __init__(self, render_mode='human'):
        super(GameEnv, self).__init__()

        # defining the screen dimension for render purpose
        self.screen_width = SCREEN_WIDTH
        self.screen_height = SCREEN_HEIGHT
        self.render_mode = render_mode

        total_values = 219
        self.observation_space = Box(
            low=np.zeros(total_values, dtype=np.float32),
            high=self.screen_width * np.ones(total_values, dtype=np.float32),
            dtype=np.float32
        )

        # Define the radius for the semi-circular reward area
        self.semi_circle_radius = 250  # You can adjust this radius

        #! Define the center of the screen for the green dot
        self.center_x = self.screen_width // 2
        self.center_y = self.screen_height // 2

        self.action_space = Discrete(5)

        self.total_steps = 0

        self.number_of_predator = 1

        self.predator_agent = None

        self.predator_total_reward = 0

        self.obs = None

        # start the tick timer
        self.start_time = 0
        self.total_running_time = 10

        pygame.init()
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption('Multi Agent Environment(simple)')

        pygame.font.init()
        self.font = pygame.font.Font(None, 18)

        self.wall = Walls(pygame)
        self.walls = None

    def agent_init(self):
        predator_agents = Agent('predator', 0)
        self.predator_agent = predator_agents

    def flatten_list(self, nested_list):
        flattened_list = []
        for item in nested_list:
            if isinstance(item, list):
                flattened_list.extend(self.flatten_list(item))
            else:
                flattened_list.append(item)
        return flattened_list
    
    def _get_obs(self):
        observation = []
        agent_pos = [self.predator_agent.current_position[0], self.predator_agent.current_position[1]]
        observation.append(agent_pos)

        angle = self.predator_agent.angle
        observation.append(angle)

        value_list = get_fov_rays(agent_pos)
        observation.append(value_list)
        
        observation = self.flatten_list(observation)
        return observation

    def _max_right(self):
        max_right = 0
        for wall in self.walls:
            if wall.right > max_right:
                max_right = wall.right
        return max_right

    def reset(self, seed=0):
        self.start_time = time.time()
        self.agent_init()
        self.wall.clear_walls()
        self.walls = self.wall.make_wall(WALLS2)
        self.total_steps = 0
        self.predator_total_reward = 0

        predator = self.predator_agent
        predator.agent_reset(width=self.screen_width, height=self.screen_height, walls=self.walls)

        self.predator_agent = predator

        observation = self._get_obs()
        self.obs = observation

        return observation, seed

    def step(self, action):
        done = False
        reward = 0
        truncated = False
        info = {}
        current_time = time.time()

        elapsed_time = current_time - self.start_time
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                pygame.quit()
        self.predator_agent.step_update(action, range_x=self.screen_width, range_y=self.screen_height)
        self.predator_agent = detect_collision(self.predator_agent, self.walls)

        # Calculate the center of the exponential reward area
        center_x, center_y = 325, 300
        # Calculate the distance from the agent to the center
        distance_to_center = ((self.predator_agent.current_position[0] - center_x) ** 2 + (self.predator_agent.current_position[1] - center_y) ** 2) ** 0.5
        # Set the radius for the exponential reward function to start working
        radius_for_exponential_reward = 100  # Adjust this radius as needed
        # Check if the agent is within the exponential reward area
        if distance_to_center <= radius_for_exponential_reward:
            reward = 10 * np.exp(-1 * distance_to_center / 100) + 0.05

        if self.predator_agent.current_position[0] > self._max_right():
            reward += 100
            done = True

        if elapsed_time >= self.total_running_time:
            reward -= 50
            done = True

        reward += 0.01
        self.render()

        observation = self._get_obs()
        self.predator_total_reward = reward
        self.obs = observation

        return observation, reward, done, truncated, info

    def render(self):
        if self.render_mode == 'human':
            screen = self.screen
            screen.fill(WHITE)
            
            predator = self.predator_agent
            pygame.draw.circle(screen, RED, predator.center, predator.radius)
            pygame.draw.line(screen, RED, predator.center, predator.draw_direction_end, 5)

            for key, wall in WALLS2.items():
                pygame.draw.rect(screen, BLUE, (wall['x'], wall['y'], wall['width'], wall['height']))

            max_right = self._max_right()
            pygame.draw.circle(screen, (0, 255, 0), (325, 300), 5)

            max_right_color = (255, 0, 0)
            pygame.draw.line(self.screen, max_right_color, (max_right, 0), (max_right, self.screen_height), 2)

            pygame.display.update()

    def close(self):
        pygame.quit()


In [ ]:
env = GameEnv()

In [ ]:
res = env.reset()

In [ ]:
from stable_baselines3 import PPO

In [ ]:
log_path = os.path.join('Training', 'Logs', 'Level_01_PPO')
baseline_path = os.path.join('Training', 'Models', 'Level_01_PPO')

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
env.reset()
model.learn(total_timesteps=50000)

In [ ]:
env.close()

In [ ]:
model.save(baseline_path)


In [ ]:
%tensorboard

In [ ]:
import os
import sys
import time
import numpy as np
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Box

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from Agents.agent import Agent
from Agents.RayCast import get_fov_rays
from Constants.constants import WHITE, RED, BLUE, SCREEN_WIDTH, SCREEN_HEIGHT, WALLS, WALLS2
from Walls.collision_detection import detect_collision
from Walls.wall_class import Walls

class GameEnv(Env):
    def __init__(self, render_mode='human'):
        super(GameEnv, self).__init__()

        # defining the screen dimension for render purpose
        self.screen_width = SCREEN_WIDTH
        self.screen_height = SCREEN_HEIGHT
        self.render_mode = render_mode

        total_values = 219
        self.observation_space = Box(
            low=np.zeros(total_values, dtype=np.float32),
            high=self.screen_width * np.ones(total_values, dtype=np.float32),
            dtype=np.float32
        )

        # Define the radius for the semi-circular reward area
        self.semi_circle_radius = 250  # You can adjust this radius

        #! Define the center of the screen for the green dot
        self.center_x = self.screen_width // 2
        self.center_y = self.screen_height // 2

        self.action_space = Discrete(5)

        self.total_steps = 0

        self.number_of_predator = 1

        self.predator_agent = None

        self.predator_total_reward = 0

        self.obs = None

        # start the tick timer
        self.start_time = 0
        self.total_running_time = 10

        pygame.init()
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption('Multi Agent Environment(simple)')

        pygame.font.init()
        self.font = pygame.font.Font(None, 18)

        self.wall = Walls(pygame)
        self.walls = None

    def agent_init(self):
        predator_agents = Agent('predator', 0)
        self.predator_agent = predator_agents

    def flatten_list(self, nested_list):
        flattened_list = []
        for item in nested_list:
            if isinstance(item, list):
                flattened_list.extend(self.flatten_list(item))
            else:
                flattened_list.append(item)
        return flattened_list
    
    def _get_obs(self):
        observation = []
        agent_pos = [self.predator_agent.current_position[0], self.predator_agent.current_position[1]]
        observation.append(agent_pos)

        angle = self.predator_agent.angle
        observation.append(angle)

        value_list = get_fov_rays(agent_pos)
        observation.append(value_list)
        
        observation = self.flatten_list(observation)
        return observation

    def _max_right(self):
        max_right = 0
        for wall in self.walls:
            if wall.right > max_right:
                max_right = wall.right
        return max_right

    def reset(self, seed=0):
        self.start_time = time.time()
        self.agent_init()
        self.wall.clear_walls()
        self.walls = self.wall.make_wall(WALLS2)
        self.total_steps = 0
        self.predator_total_reward = 0

        predator = self.predator_agent
        predator.agent_reset(width=self.screen_width, height=self.screen_height, walls=self.walls)

        self.predator_agent = predator

        observation = self._get_obs()
        self.obs = observation

        return observation, seed

    def step(self, action):
        done = False
        reward = 0
        truncated = False
        info = {}
        current_time = time.time()

        elapsed_time = current_time - self.start_time
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                pygame.quit()
        self.predator_agent.step_update(action, range_x=self.screen_width, range_y=self.screen_height)
        self.predator_agent = detect_collision(self.predator_agent, self.walls)

        self.total_steps += 1
        # Calculate the distance to the center (325, 300)
        center_x = 325
        center_y = 300
        distance_to_center = ((self.predator_agent.current_position[0] - center_x) ** 2 + (self.predator_agent.current_position[1] - center_y) ** 2) ** 0.5
        # Use the given exponential reward function
        reward = 100 * np.exp(-2 * distance_to_center / 100) - 1.5
         
        if self.predator_agent.current_position[0] > self._max_right():
            reward += 100
            done = True

        if elapsed_time >= self.total_running_time:
            reward -= 50
            done = True
      
        reward += 0.01
        self.render()

        observation = self._get_obs()
        self.predator_total_reward = reward
        self.obs = observation

        return observation, reward, done, truncated, info

    def render(self):
        if self.render_mode == 'human':
            screen = self.screen
            screen.fill(WHITE)
            
            predator = self.predator_agent
            pygame.draw.circle(screen, RED, predator.center, predator.radius)
            pygame.draw.line(screen, RED, predator.center, predator.draw_direction_end, 5)

            for key, wall in WALLS2.items():
                pygame.draw.rect(screen, BLUE, (wall['x'], wall['y'], wall['width'], wall['height']))


            # Calculate the center of the exponential reward area
            center_x, center_y = 325, 300
            # Calculate the distance from the agent to the center
            distance_to_center = ((self.predator_agent.current_position[0] - center_x) ** 2 + (self.predator_agent.current_position[1] - center_y) ** 2) ** 0.5
            # Check if the agent is within the exponential reward area
            if distance_to_center <= 250:  # Adjust this radius as needed
                reward = 100 * np.exp(-2 * distance_to_center / 100) - 1.5


            max_right = self._max_right()
            pygame.draw.circle(screen, (0, 255, 0), (325, 300), 5)

            max_right_color = (255, 0, 0)
            pygame.draw.line(self.screen, max_right_color, (max_right, 0), (max_right, self.screen_height), 2)

            pygame.display.update()

    def close(self):
        pygame.quit()

def main():
    # Initialize the environment
    env = GameEnv(render_mode='human')

    # Reset the environment to get the initial observation
    observation, seed = env.reset()

    done = False
    while not done:
        # Replace this with your agent's logic to choose actions
        # In this example, we're taking a random action.
        action = env.action_space.sample()

        # Step through the environment with the chosen action
        observation, reward, done, truncated, info = env.step(action)

    # Close the environment when done
    env.close()

if __name__ == "__main__":
    main()


In [ ]:
env.close()

In [ ]:
pygame.quit()

# SPECIAL WORKS


In [ ]:
import pygame
import sys
import math

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10
move_step = 1

# Define objects with arbitrary shapes
objects = {
    "special_wall": [(100, 100), (120, 100), (120, 200), (100, 200)],
    "wall2": [(200, 300), (220, 300), (220, 400), (200, 400)],
    "wall3": [(300, 200), (320, 200), (320, 300), (300, 300)],
    "circular_wall": [(400, 400), (430, 400), (430, 430), (400, 430)]
}

def is_ray_blocked(ray_start, ray_angle):
    closest_intersection = None

    x1, y1 = ray_start
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

    for object_name, object_vertices in objects.items():
        for i in range(len(object_vertices)):
            x3, y3 = object_vertices[i]
            x4, y4 = object_vertices[(i + 1) % len(object_vertices)]

            denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if denominator == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

            epsilon = 1e-5  # Small epsilon value

            if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
                intersection_x = x1 + t * (x2 - x1)
                intersection_y = y1 + t * (y2 - y1)

                distance = math.hypot(intersection_x - x1, intersection_y - y1)
                if closest_intersection is None or distance < closest_intersection:
                    closest_intersection = distance

    return closest_intersection

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        predator_x -= move_step
    if keys[pygame.K_RIGHT]:
        predator_x += move_step
    if keys[pygame.K_UP]:
        predator_y -= move_step
    if keys[pygame.K_DOWN]:
        predator_y += move_step

    screen.fill(WHITE)

    for angle in range(360):
        ray_start = (predator_x, predator_y)
        ray_angle = angle
        closest_intersection = is_ray_blocked(ray_start, ray_angle)

        if closest_intersection is not None:
            ray_end_x = ray_start[0] + closest_intersection * math.cos(math.radians(ray_angle))
            ray_end_y = ray_start[1] + closest_intersection * math.sin(math.radians(ray_angle))
            pygame.draw.line(screen, RED, ray_start, (ray_end_x, ray_end_y), 1)
        else:
            ray_end_x = ray_start[0] + 1000 * math.cos(math.radians(ray_angle))
            ray_end_y = ray_start[1] + 1000 * math.sin(math.radians(ray_angle))
            pygame.draw.line(screen, BLUE, ray_start, (ray_end_x, ray_end_y), 1)

    pygame.draw.circle(screen, RED, (predator_x, predator_y), predator_radius)

    for object_name, object_vertices in objects.items():
        pygame.draw.polygon(screen, GREEN, object_vertices)
        

    pygame.display.flip()

pygame.quit()
sys.exit()


In [ ]:
import pygame
import sys
import math

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)  # Define green color
BLUE = (0, 0, 255)   # Define blue color for vision rays

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10  # Radius of the circular predator agent
move_step = 1  # Adjust the movement step

# Define walls in a dictionary
walls = {
    "special_wall": {"x": 100, "y": 100, "width": 20, "height": 100},
    "wall2": {"x": 200, "y": 300, "width": 20, "height": 100},
    "wall3": {"x": 300, "y": 200, "width": 20, "height": 100},
}

# Function to check ray-wall intersection
def is_ray_blocked(ray_start, ray_angle):
    closest_intersection = None
    blocking_wall = None

    x1, y1 = ray_start
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

    for wall_name, wall in walls.items():
        x3, y3 = wall["x"], wall["y"]
        x4, y4 = x3 + wall["width"], y3 + wall["height"]

        denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

        if denominator == 0:
            continue

        t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
        u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

        if 0 <= t <= 1 and 0 <= u <= 1:
            intersection_x = x1 + t * (x2 - x1)
            intersection_y = y1 + t * (y2 - y1)

            if not closest_intersection or t < closest_intersection:
                closest_intersection = t
                blocking_wall = wall_name

    return closest_intersection, blocking_wall

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Handle user input
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        predator_x -= move_step
    if keys[pygame.K_RIGHT]:
        predator_x += move_step
    if keys[pygame.K_UP]:
        predator_y -= move_step
    if keys[pygame.K_DOWN]:
        predator_y += move_step

    # Clear the screen
    screen.fill(WHITE)

    # Draw vision rays
    for angle in range(360):
        ray_start = (predator_x, predator_y)
        ray_angle = angle
        ray_end = (ray_start[0] + 1000 * math.cos(math.radians(ray_angle)),
                   ray_start[1] + 1000 * math.sin(math.radians(ray_angle)))

        # Check if the ray is blocked by any wall
        closest_intersection, blocking_wall = is_ray_blocked(ray_start, ray_angle)

        if closest_intersection is not None:
            ray_end_x = ray_start[0] + closest_intersection * (ray_end[0] - ray_start[0])
            ray_end_y = ray_start[1] + closest_intersection * (ray_end[1] - ray_start[1])

            if blocking_wall == "special_wall":
                color = RED
            else:
                color = GREEN

            pygame.draw.line(screen, color, ray_start, (ray_end_x, ray_end_y), 1)
        else:
            pygame.draw.line(screen, BLUE, ray_start, ray_end, 1)

    # Draw the predator agent as a circle
    pygame.draw.circle(screen, RED, (predator_x, predator_y), predator_radius)

    # Draw walls from the dictionary with green color
    for wall in walls.values():
        pygame.draw.rect(screen, GREEN, (wall["x"], wall["y"], wall["width"], wall["height"]))

    # Update the display
    pygame.display.flip()

# Quit Pygame
pygame.quit()
sys.exit()


In [ ]:
import pygame
import sys
import math

class GameEnv:
    def __init__(self, width, height):
        pygame.init()
        self.WIDTH = width
        self.HEIGHT = height
        self.screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption("Predator Agent Environment")

        # Define colors
        self.RED = (255, 0, 0)
        self.WHITE = (255, 255, 255)
        self.GREEN = (0, 255, 0)
        self.BLUE = (0, 0, 255)

        # Predator agent position and size
        self.predator_x, self.predator_y = self.WIDTH // 2, self.HEIGHT // 2
        self.predator_radius = 10
        self.move_step = 1

        # Define objects with arbitrary shapes
        self.objects = {
            "special_wall": [(100, 100), (120, 100), (120, 200), (100, 200)],
            "wall2": [(200, 300), (220, 300), (220, 400), (200, 400)],
            "wall3": [(300, 200), (320, 200), (320, 300), (300, 300)],
            "circular_wall": [(400, 400), (430, 400), (430, 430), (400, 430)]
        }

    def is_ray_blocked(self, ray_start, ray_angle):
        closest_intersection = None

        x1, y1 = ray_start
        x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

        for object_name, object_vertices in self.objects.items():
            for i in range(len(object_vertices)):
                x3, y3 = object_vertices[i]
                x4, y4 = object_vertices[(i + 1) % len(object_vertices)]

                denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

                if denominator == 0:
                    continue

                t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
                u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

                epsilon = 1e-5  # Small epsilon value

                if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
                    intersection_x = x1 + t * (x2 - x1)
                    intersection_y = y1 + t * (y2 - y1)

                    distance = math.hypot(intersection_x - x1, intersection_y - y1)
                    if closest_intersection is None or distance < closest_intersection:
                        closest_intersection = distance

        return closest_intersection

    def render(self):
        self.screen.fill(self.WHITE)

        for angle in range(360):
            ray_start = (self.predator_x, self.predator_y)
            ray_angle = angle
            closest_intersection = self.is_ray_blocked(ray_start, ray_angle)

            if closest_intersection is not None:
                ray_end_x = ray_start[0] + closest_intersection * math.cos(math.radians(ray_angle))
                ray_end_y = ray_start[1] + closest_intersection * math.sin(math.radians(ray_angle))
                pygame.draw.line(self.screen, self.RED, ray_start, (ray_end_x, ray_end_y), 1)
            else:
                ray_end_x = ray_start[0] + 1000 * math.cos(math.radians(ray_angle))
                ray_end_y = ray_start[1] + 1000 * math.sin(math.radians(ray_angle))
                pygame.draw.line(self.screen, self.BLUE, ray_start, (ray_end_x, ray_end_y), 1)

        pygame.draw.circle(self.screen, self.RED, (self.predator_x, self.predator_y), self.predator_radius)

        for object_name, object_vertices in self.objects.items():
            pygame.draw.polygon(self.screen, self.GREEN, object_vertices)

        pygame.display.flip()

    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT]:
                self.predator_x -= self.move_step
            if keys[pygame.K_RIGHT]:
                self.predator_x += self.move_step
            if keys[pygame.K_UP]:
                self.predator_y -= self.move_step
            if keys[pygame.K_DOWN]:
                self.predator_y += self.move_step

            self.render()

        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = GameEnv(800, 600)
    game.run()


In [ ]:
import pygame
import sys
import math

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10
move_step = 1

# Define objects with arbitrary shapes
objects = {
    # "wall1": [(100, 100), (120, 100), (120, 200), (100, 200)],
    "wall2": [(200, 300), (220, 300), (220, 400), (200, 400)],
    "wall3": [(300, 200), (320, 200), (320, 300), (300, 300)],
    "circular_wall": [(400, 400), (430, 400), (430, 430), (400, 430)],
    "special_point": (100, 100)  # This is now a single point
}

# Function to check ray-object intersection
def does_agent_see_special_point(ray_start, ray_angle, special_point, objects):
    x1, y1 = ray_start
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

    for object_name, object in objects.items():
        if object_name == "special_point":
            continue  # Skip the special_point, as it's not a polygon

        for i in range(len(object)):
            x3, y3 = object[i]
            x4, y4 = object[(i + 1) % len(object)]

            denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if denominator == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersection_x = x1 + t * (x2 - x1)
                intersection_y = y1 + t * (y2 - y1)

                # Check if the intersection point is the special point
                if (abs(intersection_x - special_point[0]) < 1 and
                        abs(intersection_y - special_point[1]) < 1):
                    return True  # Agent sees the special point

    return False  # Agent does not see the special point

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        predator_x -= move_step
    if keys[pygame.K_RIGHT]:
        predator_x += move_step
    if keys[pygame.K_UP]:
        predator_y -= move_step
    if keys[pygame.K_DOWN]:
        predator_y += move_step

    screen.fill(WHITE)

    for angle in range(360):
        ray_start = (predator_x, predator_y)
        ray_angle = angle

        sees_special_point = does_agent_see_special_point(ray_start, ray_angle, objects["special_point"], objects)

        ray_end = (ray_start[0] + 1000 * math.cos(math.radians(ray_angle)),
                   ray_start[1] + 1000 * math.sin(math.radians(ray_angle)))

        if sees_special_point:
            color = RED
        else:
            color = BLUE

        pygame.draw.line(screen, color, ray_start, ray_end, 1)

    pygame.draw.circle(screen, RED, (predator_x, predator_y), predator_radius)

    for object_name, object in objects.items():
        if object_name == "special_point":
            continue
        pygame.draw.polygon(screen, GREEN, object)

    pygame.display.flip()

pygame.quit()
sys.exit()


In [ ]:
pygame.quit()


In [ ]:
import pygame
import sys
import math

class GameEnv:
    def __init__(self, width, height):
        pygame.init()
        self.WIDTH = width
        self.HEIGHT = height
        self.screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption("Predator Agent Environment")

        # Define colors
        self.RED = (255, 0, 0)
        self.WHITE = (255, 255, 255)
        self.GREEN = (0, 255, 0)
        self.BLUE = (0, 0, 255)

        # Predator agent position and size
        self.predator_x, self.predator_y = self.WIDTH // 2, self.HEIGHT // 2
        self.predator_radius = 10
        self.move_step = 1

        self.special_point = (100, 100)  # Single point at (100, 100)
        # Define objects with arbitrary shapes
        self.objects = {
            "wall2": [(200, 300), (220, 300), (220, 400), (200, 400)],
            "wall3": [(300, 200), (320, 200), (320, 300), (300, 300)],
            "wall4": [(400, 400), (430, 400), (430, 430), (400, 430)]
        }

    def is_ray_blocked(self, ray_start, ray_angle):
        point_visible = True  # Assume the point is initially visible

        x1, y1 = ray_start
        x2, y2 = self.special_point  # Endpoint of the ray is the special point

        for object_name, object_vertices in self.objects.items():
            for i in range(len(object_vertices)):
                x3, y3 = object_vertices[i]
                x4, y4 = object_vertices[(i + 1) % len(object_vertices)]

                denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

                if denominator == 0:
                    continue

                t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
                u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

                epsilon = 1e-5  # Small epsilon value
                if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
                    point_visible = False
                    break

        return point_visible

    def render(self):
        self.screen.fill(self.WHITE)

        ray_start = (self.predator_x, self.predator_y)
        ray_angle = math.degrees(math.atan2(self.special_point[1] - self.predator_y, self.special_point[0] - self.predator_x))

        # Draw the dynamic ray
        ray_end_x = self.special_point[0]
        ray_end_y = self.special_point[1]
        ray_obstructed = self.is_ray_blocked(ray_start, ray_angle)
    
        if ray_obstructed:
            pygame.draw.line(self.screen, self.RED, ray_start, (ray_end_x, ray_end_y), 1)

        # Draw the special point
        pygame.draw.circle(self.screen, self.RED, self.special_point, 5)

        for object_name, object_vertices in self.objects.items():
            pygame.draw.polygon(self.screen, self.GREEN, object_vertices)

        # Draw the predator agent
        pygame.draw.circle(self.screen, self.RED, (self.predator_x, self.predator_y), self.predator_radius)

        pygame.display.flip()



    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT]:
                self.predator_x -= self.move_step
            if keys[pygame.K_RIGHT]:
                self.predator_x += self.move_step
            if keys[pygame.K_UP]:
                self.predator_y -= self.move_step
            if keys[pygame.K_DOWN]:
                self.predator_y += self.move_step

            self.render()

        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = GameEnv(800, 600)
    game.run()


# POINT RAY START

In [ ]:
import pygame
import numpy as np

from Walls.wall_class import Walls
from Walls.collision_detection import detect_collision
from Agents.agent import Agent
from Constants.constants import LEVEL_4_WALLS, SCREEN_WIDTH, SCREEN_HEIGHT

In [ ]:
pygame.init()
pygame.display.init()
window = pygame.display.set_mode((SCREEN_HEIGHT, SCREEN_WIDTH))
clock = pygame.time.Clock()
screen = pygame.Surface((SCREEN_HEIGHT, SCREEN_WIDTH))

In [ ]:
wall = Walls(pygame)
walls = wall.make_wall(LEVEL_4_WALLS)

In [ ]:
walls


In [ ]:
agent = Agent("zahin",0)
agent.agent_reset(SCREEN_WIDTH,SCREEN_HEIGHT, walls)

In [ ]:
def is_ray_blocked(agent_pos, goal, walls):
        point_visible = True  # Assume the point is initially visible
        # list 
        ray_start = agent_pos
        x1, y1 = ray_start
        x2, y2 = goal  # Endpoint of the ray is the special point

        for wall in walls:
            wall_vertices = [wall.topleft, wall.topright, wall.bottomright, wall.bottomleft]
            for i in range(len(wall_vertices)):
                x3, y3 = wall_vertices[i]
                x4, y4 = wall_vertices[(i + 1) % len(wall_vertices)]

                denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

                if denominator == 0:
                    continue

                t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
                u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

                epsilon = 1e-5  # Small epsilon value
                if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
                    point_visible = False
                    break

        return point_visible


In [ ]:
def render():
    screen.fill((255, 255, 255))

    pygame.draw.circle(screen, (255, 0, 0), agent.center, agent.radius)
    pygame.draw.line(screen, (255, 0, 0), agent.center, agent.draw_direction_end, 5)

    agent_pos = agent.center
    goal = (750, 50)
    pygame.draw.circle(screen, (0, 255, 255), goal, 20)
    if is_ray_blocked(agent_pos, goal, walls):
      pygame.draw.line(screen, (255, 0, 255), agent_pos, goal, 5) 

    for key, wall in LEVEL_4_WALLS.items():
       pygame.draw.rect(screen, (255, 255, 0), (wall['x'], wall['y'], wall['width'], wall['height']))
    
    pygame.draw.rect(screen, (0, 0, 0), (650, 0, 150, 150), 2)
    
    window.blit(screen, screen.get_rect())
    pygame.event.pump()
    pygame.display.update()
    clock.tick(60)



In [ ]:
pygame.init()
pygame.display.init()
window = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()
screen = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))
# 130 degree angle, 10 degree por por. return angle and length as list.
running = True
range_x = 770
range_y = 570

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    key = pygame.key.get_pressed()
    if key[pygame.K_UP]:
        agent.step_update(2, range_x, range_y)
    if key[pygame.K_LEFT]:
        agent.step_update(1, range_x, range_y)
    if key[pygame.K_RIGHT]:
        agent.step_update(0, range_x, range_y)
    agent = detect_collision(agent, walls)

    render()
pygame.display.quit()
pygame.quit()


In [ ]:
pygame.quit()

# POINT RAY ENDS

In [ ]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")
import os
import sys
import time
import numpy as np
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Box

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from Agents.agent import Agent
from Agents.RayCast import get_fov_rays
from Constants.constants import WHITE, RED, BLUE, SCREEN_WIDTH, SCREEN_HEIGHT,  LEVEL_4_WALLS
from Walls.collision_detection import detect_collision
from Walls.wall_class import Walls



class GameEnv(Env):
    def __init__(self, render_mode='human'):
        super(GameEnv, self).__init__()

        # defining the screen dimension for render purpose
        self.screen_width = SCREEN_WIDTH
        self.screen_height = SCREEN_HEIGHT
        self.render_mode = render_mode

        total_values = 219
        self.observation_space = Box(
            low=np.zeros(total_values, dtype=np.float32),
            high=self.screen_width * np.ones(total_values, dtype=np.float32),
            dtype=np.float32
        )

        # Define the radius for the semi-circular reward area
        self.semi_circle_radius = 250  # You can adjust this radius

        #! Define the center of the screen for the green dot
        self.center_x = self.screen_width // 2
        self.center_y = self.screen_height // 2

        self.action_space = Discrete(5)

        self.total_steps = 0

        self.number_of_predator = 1

        self.predator_agent = None

        self.predator_total_reward = 0

        self.obs = None

        # start the tick timer
        self.start_time = 0
        self.total_running_time = 10

        pygame.init()
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption('Multi Agent Environment(simple)')

        pygame.font.init()
        self.font = pygame.font.Font(None, 18)

        self.wall = Walls(pygame)
        self.walls = None

    def agent_init(self):
        predator_agents = Agent('predator', 0)
        self.predator_agent = predator_agents

    def flatten_list(self, nested_list):
        flattened_list = []
        for item in nested_list:
            if isinstance(item, list):
                flattened_list.extend(self.flatten_list(item))
            else:
                flattened_list.append(item)
        return flattened_list
    
    def _get_obs(self):
        observation = []
        agent_pos = [self.predator_agent.current_position[0], self.predator_agent.current_position[1]]
        observation.append(agent_pos)

        angle = self.predator_agent.angle
        observation.append(angle)

        value_list = get_fov_rays(agent_pos)
        observation.append(value_list)
        
        observation = self.flatten_list(observation)
        return observation

    def _max_right(self):
        max_right = 0
        for wall in self.walls:
            if wall.right > max_right:
                max_right = wall.right
        return max_right

    def reset(self, seed=0):
        self.start_time = time.time()
        self.agent_init()
        self.wall.clear_walls()
        self.walls = self.wall.make_wall(LEVEL_3_WALLS)
        self.total_steps = 0
        self.predator_total_reward = 0

        predator = self.predator_agent
        predator.agent_reset(width=self.screen_width, height=self.screen_height, walls=self.walls)

        self.predator_agent = predator

        observation = self._get_obs()
        self.obs = observation

        return observation, seed

    def step(self, action):
        done = False
        reward = 0
        truncated = False
        info = {}
        current_time = time.time()

        elapsed_time = current_time - self.start_time
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                pygame.quit()
        self.predator_agent.step_update(action, range_x=self.screen_width, range_y=self.screen_height)
        self.predator_agent = detect_collision(self.predator_agent, self.walls)

        # Calculate the center of the exponential reward area
        center_x, center_y = 325, 300
        # Calculate the distance from the agent to the center
        distance_to_center = ((self.predator_agent.current_position[0] - center_x) ** 2 + (self.predator_agent.current_position[1] - center_y) ** 2) ** 0.5
        # Set the radius for the exponential reward function to start working
        radius_for_exponential_reward = 250  # Adjust this radius as needed
        # Check if the agent is within the exponential reward area
        if distance_to_center <= radius_for_exponential_reward:
            reward = 10 * np.exp(-1 * distance_to_center / 100) + 0.05

        if self.predator_agent.current_position[0] > self._max_right():
            reward += 100
            done = True

        if elapsed_time >= self.total_running_time:
            reward -= 50
            done = True

        # reward += 0.01
        self.render()

        observation = self._get_obs()
        self.predator_total_reward = reward
        self.obs = observation

        return observation, reward, done, truncated, info

    def render(self):
        if self.render_mode == 'human':
            screen = self.screen
            screen.fill(WHITE)
            
            predator = self.predator_agent
            pygame.draw.circle(screen, RED, predator.center, predator.radius)
            pygame.draw.line(screen, RED, predator.center, predator.draw_direction_end, 5)

            for key, wall in LEVEL_3_WALLS.items():
                pygame.draw.rect(screen, BLUE, (wall['x'], wall['y'], wall['width'], wall['height']))


            # Calculate the center of the exponential reward area
            center_x, center_y = 325, 300
            # Calculate the distance from the agent to the center
            distance_to_center = ((self.predator_agent.current_position[0] - center_x) ** 2 + (self.predator_agent.current_position[1] - center_y) ** 2) ** 0.5
            # Check if the agent is within the exponential reward area
            if distance_to_center <= 250:  # Adjust this radius as needed
                reward = 100 * np.exp(-2 * distance_to_center / 100) - 1.5


            max_right = self._max_right()
            pygame.draw.circle(screen, (0, 255, 0), (325, 300), 5)

            max_right_color = (255, 0, 0)
            pygame.draw.line(self.screen, max_right_color, (max_right, 0), (max_right, self.screen_height), 2)

            pygame.display.update()

    def close(self):
        pygame.quit()

def main():
    # Initialize the environment
    env = GameEnv(render_mode='human')

    # Reset the environment to get the initial observation
    observation, seed = env.reset()

    done = False
    while not done:
        # Replace this with your agent's logic to choose actions
        # In this example, we're taking a random action.
        action = env.action_space.sample()

        # Step through the environment with the chosen action
        observation, reward, done, truncated, info = env.step(action)

    # Close the environment when done
    env.close()

if __name__ == "__main__":
    main()

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)  # Define green color

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10  # Radius of the circular predator agent
move_step = 0.1  # Adjust the movement step

# Define walls in a dictionary
walls = {
    # "wall1": {"x": 100, "y": 100, "width": 20, "height": 100},
    "wall2": {"x": 200, "y": 300, "width": 20, "height": 100},
    "wall3": {"x": 300, "y": 200, "width": 20, "height": 100},
}

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Handle user input
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        predator_x -= move_step
    if keys[pygame.K_RIGHT]:
        predator_x += move_step
    if keys[pygame.K_UP]:
        predator_y -= move_step
    if keys[pygame.K_DOWN]:
        predator_y += move_step

    # Clear the screen
    screen.fill(WHITE)

    # Draw the predator agent as a circle
    pygame.draw.circle(screen, RED, (predator_x, predator_y), predator_radius)

    # Draw walls from the dictionary with green color
    for wall in walls.values():
        pygame.draw.rect(screen, GREEN, (wall["x"], wall["y"], wall["width"], wall["height"]))

    # Update the display
    pygame.display.flip()

# Quit Pygame
pygame.quit()
sys.exit()


In [ ]:
import pygame
import math
import numpy as np

from Walls.wall_class import Walls
from Agents.agent import Agent
from Constants.constants import LEVEL_4_WALLS, SCREEN_WIDTH, SCREEN_HEIGHT

# Initialize pygame
pygame.init()
pygame.display.init()
window = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()
screen = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create walls
wall = Walls(pygame)
walls = wall.make_wall(LEVEL_4_WALLS)

# Create the agent
agent = Agent("shanila", 15)
agent.agent_reset(SCREEN_WIDTH, SCREEN_HEIGHT, walls)

def cast_rays(agent, walls):
    start_angle = agent.angle - 65  # 65 degrees to the left
    end_angle = agent.angle + 65  # 65 degrees to the right
    angle_step = 130 / 13  # 13 rays in total

    ray_info = []

    for angle in np.arange(start_angle, end_angle + angle_step, angle_step):
        ray_direction = np.array([math.cos(math.radians(angle)), math.sin(math.radians(angle))])
        ray_direction /= np.linalg.norm(ray_direction)  # Normalize the direction vector

        length = float('inf')  # Initialize length to infinity

        for wall in walls:
            x1, y1 = agent.current_position
            x2, y2 = agent.current_position + length * ray_direction
            x3, y3, x4, y4 = wall['x'], wall['y'], wall['x'] + wall['width'], wall['y'] + wall['height']

            den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if den == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / den
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / den

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersection_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
                wall_length = np.linalg.norm(intersection_point - agent.current_position)
                if wall_length < length:
                    length = wall_length  # Update length if this intersection point is closer

        ray_info.append((angle, length))

    return ray_info


def render():
    screen.fill((255, 255, 255))
    agent_pos = (int(agent.current_position[0]), int(agent.current_position[1]))
    pygame.draw.circle(screen, (255, 0, 0), agent_pos, agent.radius)
    ray_info = cast_rays(agent, LEVEL_4_WALLS.values())
    for angle, length in ray_info:
        end_point = None
        if math.isfinite(length):
            end_point = (
                int(agent.current_position[0] + length * math.cos(math.radians(angle))),
                int(agent.current_position[1] + length * math.sin(math.radians(angle)))
            )
        else:
            # If length is infinite, draw the ray in the direction it was cast
            end_point = (
                int(agent.current_position[0] + SCREEN_WIDTH * math.cos(math.radians(angle))),
                int(agent.current_position[1] + SCREEN_HEIGHT * math.sin(math.radians(angle)))
            )
        pygame.draw.line(screen, (0, 0, 255), agent_pos, end_point, 2)
    for wall in LEVEL_4_WALLS.values():
        pygame.draw.rect(screen, (255, 255, 0), (wall['x'], wall['y'], wall['width'], wall['height']))
    pygame.draw.rect(screen, (0, 0, 0), (650, 0, 150, 150), 2)
    window.blit(screen, screen.get_rect())
    pygame.event.pump()
    pygame.display.update()
    clock.tick(60)



running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    key = pygame.key.get_pressed()
    if key[pygame.K_UP]:
        agent.step_update(2, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_LEFT]:
        agent.step_update(1, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_RIGHT]:
        agent.step_update(0, SCREEN_WIDTH, SCREEN_HEIGHT)

    render()

pygame.display.quit()
pygame.quit()


In [93]:
import pygame
import math
import numpy as np

from Walls.wall_class import Walls
from Agents.agent import Agent
from Constants.constants import LEVEL_4_WALLS, SCREEN_WIDTH, SCREEN_HEIGHT

# Initialize pygame
pygame.init()
pygame.display.init()
window = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()
screen = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create walls
wall = Walls(pygame)
walls = wall.make_wall(LEVEL_4_WALLS)

# Create the agent
agent = Agent("shanila", 15)
agent.agent_reset(SCREEN_WIDTH, SCREEN_HEIGHT, walls)

def cast_rays(agent, num_rays):
    start_angle = agent.angle - 65  # 65 degrees to the left
    end_angle = agent.angle + 65  # 65 degrees to the right
    angle_step = 130 / (num_rays - 1) if num_rays > 1 else 0  # Adjust the angle step for the number of rays

    ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()  # Convert to a Python list
    return ray_angles

def update_ray_lengths(agent, ray_angles, walls):
    ray_lengths = []

    for angle in ray_angles:
        ray_direction = np.array([math.cos(math.radians(angle)), math.sin(math.radians(angle))])
        ray_direction /= np.linalg.norm(ray_direction)  # Normalize the direction vector

        length = float('inf')  # Initialize length to infinity

        for wall in walls:
            x1, y1 = agent.current_position
            x2, y2 = agent.current_position + SCREEN_WIDTH * ray_direction  # Set an initial endpoint far away

            x3, y3, x4, y4 = wall['x'], wall['y'], wall['x'] + wall['width'], wall['y'] + wall['height']

            den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if den == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / den
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / den

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersection_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
                wall_length = np.linalg.norm(intersection_point - agent.current_position)
                if wall_length < length:
                    length = wall_length  # Update length if this intersection point is closer

        if length == float('inf'):
            # If no intersections were found, set the length to a finite value (e.g., distance to screen boundary)
            length = SCREEN_WIDTH  # Adjust this value accordingly

        ray_lengths.append(length)

    return ray_lengths



def render():
    screen.fill((255, 255, 255))
    agent_pos = (int(agent.current_position[0]), int(agent.current_position[1]))
    pygame.draw.circle(screen, (255, 0, 0), agent_pos, agent.radius)
    ray_angles = cast_rays(agent, 13)
    ray_lengths = update_ray_lengths(agent, ray_angles, LEVEL_4_WALLS.values())

    for angle, length in zip(ray_angles, ray_lengths):
        if math.isfinite(length):  # Check if the length is finite (ray intersects with a wall)
            end_point = (
                int(agent.current_position[0] + length * math.cos(math.radians(angle))),
                int(agent.current_position[1] + length * math.sin(math.radians(angle)))
            )
            pygame.draw.line(screen, (0, 0, 255), agent_pos, end_point, 2)
        
    for wall in LEVEL_4_WALLS.values():
        pygame.draw.rect(screen, (255, 255, 0), (wall['x'], wall['y'], wall['width'], wall['height']))
    pygame.draw.rect(screen, (0, 0, 0), (650, 0, 150, 150), 2)
    window.blit(screen, screen.get_rect())
    pygame.display.update()
    clock.tick(60)




running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    key = pygame.key.get_pressed()
    if key[pygame.K_UP]:
        agent.step_update(2, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_LEFT]:
        agent.step_update(1, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_RIGHT]:
        agent.step_update(0, SCREEN_WIDTH, SCREEN_HEIGHT)

    render()

pygame.display.quit()
pygame.quit()


TypeError: Agent.agent_reset() takes 3 positional arguments but 4 were given

In [94]:
pygame.quit()


In [92]:
import pygame
import math
import numpy as np

from Walls.wall_class import Walls
from Agents.agent import Agent
from Constants.constants import LEVEL_4_WALLS, SCREEN_WIDTH, SCREEN_HEIGHT

# Initialize pygame
pygame.init()
pygame.display.init()
window = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()
screen = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create walls
wall = Walls(pygame)
walls = wall.make_wall(LEVEL_4_WALLS)

# Create the agent
agent = Agent("shanila", 15)
agent.agent_reset(SCREEN_WIDTH, SCREEN_HEIGHT)


def cast_and_update_rays(agent, num_rays, walls):
    start_angle = agent.angle - 65  # 65 degrees to the left
    end_angle = agent.angle + 65  # 65 degrees to the right
    angle_step = 130 / (num_rays - 1) if num_rays > 1 else 0  # Adjust the angle step for the number of rays

    ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()  # Convert to a Python list
    ray_lengths = []

    for angle in ray_angles:
        ray_direction = np.array([math.cos(math.radians(angle)), math.sin(math.radians(angle))])
        ray_direction /= np.linalg.norm(ray_direction)  # Normalize the direction vector

        length = float('inf')  # Initialize length to infinity

        for wall in walls:
            x1, y1 = agent.current_position
            x2, y2 = agent.current_position + SCREEN_WIDTH * ray_direction  # Set an initial endpoint far away

            x3, y3, x4, y4 = wall['x'], wall['y'], wall['x'] + wall['width'], wall['y'] + wall['height']

            den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if den == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / den
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / den

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersection_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
                wall_length = np.linalg.norm(intersection_point - agent.current_position)
                if wall_length < length:
                    length = wall_length  # Update length if this intersection point is closer

        if length == float('inf'):
            # If no intersections were found, set the length to a finite value (e.g., distance to the screen boundary)
            length = SCREEN_WIDTH  # Adjust this value accordingly

        ray_lengths.append(length)

    return ray_angles, ray_lengths



def render():
    screen.fill((255, 255, 255))
    agent_pos = (int(agent.current_position[0]), int(agent.current_position[1]))
    pygame.draw.circle(screen, (255, 0, 0), agent_pos, agent.radius)
    ray_angles, ray_lengths = cast_and_update_rays(agent, 13, LEVEL_4_WALLS.values())


    for angle, length in zip(ray_angles, ray_lengths):
        if math.isfinite(length):  # Check if the length is finite (ray intersects with a wall)
            end_point = (
                int(agent.current_position[0] + length * math.cos(math.radians(angle))),
                int(agent.current_position[1] + length * math.sin(math.radians(angle)))
            )
            pygame.draw.line(screen, (0, 0, 255), agent_pos, end_point, 2)
        
    for wall in LEVEL_4_WALLS.values():
        pygame.draw.rect(screen, (255, 255, 0), (wall['x'], wall['y'], wall['width'], wall['height']))
    pygame.draw.rect(screen, (0, 0, 0), (650, 0, 150, 150), 2)
    window.blit(screen, screen.get_rect())
    pygame.display.update()
    clock.tick(60)


running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    key = pygame.key.get_pressed()
    if key[pygame.K_UP]:
        agent.step_update(2, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_LEFT]:
        agent.step_update(1, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_RIGHT]:
        agent.step_update(0, SCREEN_WIDTH, SCREEN_HEIGHT)

    render()

pygame.display.quit()
pygame.quit()


In [ ]:
pygame.quit()

In [ ]:
import math
import numpy as np
import pygame
from Constants.constants import SCREEN_WIDTH, GREEN

import sys
import os

sys.path.insert(1, os.path.join(sys.path[0], '..'))


class CastRays:
    def __init__(self, screen):
        self.screen = screen
        self.agent_center = None
        self.num_rays = None
        self.ray_angles = None
        self.ray_lengths = None

    def update_cast_rays(self, agent, walls):
        # agent and walls here are objects not list or anything
        start_angle = agent.angle - 65  # 65 degrees to the left
        end_angle = agent.angle + 65  # 65 degrees to the right
        # angle_step = 130 / (num_rays - 1) if num_rays > 1 else 0  # Adjust the angle step for the number of rays
        angle_step = np.abs((start_angle - end_angle) / 13)

        ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()  # Convert to a Python list
        ray_angles = [ray % 360 for ray in ray_angles]
        self.ray_angles = ray_angles
        self.num_rays = len(ray_angles)

        ray_lengths = []

        for angle in ray_angles:

            angle = math.radians(angle)
            ray_direction = np.array([math.cos(angle), math.sin(angle)], dtype=np.float32)

            ray_direction /= np.linalg.norm(ray_direction)  # Normalize the direction vector

            length = float('inf')  # Initialize length to infinity

            for wall in walls:
                x1, y1 = agent.current_position
                self.agent_center = (int(x1), int(y1))
                x2, y2 = agent.current_position + SCREEN_WIDTH * ray_direction  # Set an initial endpoint far away

                x3, y3, x4, y4 = wall['x'], wall['y'], wall['width'], wall['height']

                den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

                if den == 0:
                    continue

                t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / den
                u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / den

                if 0 <= t <= 1 and 0 <= u <= 1:
                    intersection_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
                    # print('inside intersect')
                    wall_length = np.linalg.norm(intersection_point - agent.current_position)
                    if wall_length < length:
                        length = wall_length

            if length == float('inf'):
                length = SCREEN_WIDTH

            ray_lengths.append(length)
            self.ray_lengths = ray_lengths

        

        return ray_angles, ray_lengths

    # def draw_rays(self):
    #     # print(self.intersect_points)
    #     for ray_length, ray_angle in zip(self.ray_lengths, self.ray_angles):
    #
    #         ray_angle = math.radians(ray_angle)
    #         end_point = (int(math.cos(ray_angle) * ray_length), int(math.sin(ray_angle)) * ray_length)
    #         ray_rect = pygame.draw.line(self.screen, GREEN, self.agent_center, end_point)






# def cast_and_update_rays(agent, walls):
#     start_angle = agent.angle - 65  # 65 degrees to the left
#     end_angle = agent.angle + 65  # 65 degrees to the right
#     # angle_step = 130 / (num_rays - 1) if num_rays > 1 else 0  # Adjust the angle step for the number of rays
#     angle_step = np.abs((start_angle - end_angle) / 10)
#
#     ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()  # Convert to a Python list
#     ray_lengths = []
#
#     for angle in ray_angles:
#         ray_direction = np.array([math.cos(math.radians(angle)), math.sin(math.radians(angle))])
#         ray_direction /= np.linalg.norm(ray_direction)  # Normalize the direction vector
#
#         length = float('inf')  # Initialize length to infinity
#
#         for wall in walls:
#             x1, y1 = agent.current_position
#             x2, y2 = agent.current_position + SCREEN_WIDTH * ray_direction  # Set an initial endpoint far away
#
#             x3, y3, x4, y4 = wall.x, wall.y, wall.width, wall.height
#
#             den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
#
#             if den == 0:
#                 continue
#
#             t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / den
#             u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / den
#
#             if 0 <= t <= 1 and 0 <= u <= 1:
#                 intersection_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
#                 wall_length = np.linalg.norm(intersection_point - agent.current_position)
#                 if wall_length < length:
#                     length = wall_length
#
#         if length == float('inf'):
#             length = SCREEN_WIDTH
#         ray_lengths.append(length)
#
#     return ray_angles, ray_lengths
#

In [ ]:
pygame.quit()

In [ ]:
import pygame
import math
import numpy as np

from Walls.wall_class import Walls
from Agents.agent import Agent
from Constants.constants import  SCREEN_WIDTH, SCREEN_HEIGHT

# Initialize pygame
pygame.init()
pygame.display.init()
window = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()
screen = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))


# walls = LEVEL_4_WALLS  # Your dictionary of walls
LEVEL_4_WALLS = {
        '1': {'x': SCREEN_WIDTH // 2 - 25, 'y': 0, 'width': 50, 'height': 120, 'name': 'obs_w', 'gate_with' : 2, 'access' : 'left_right', 'orientation' : 'vertical'},  
        '2': {'x': SCREEN_WIDTH // 2 - 25, 'y': 220, 'width': 50, 'height': 240, 'name': 'obs_w', 'gate_with' : 1, 'access' : 'left_right', 'orientation' : 'vertical'}, 
        '3': {'x': SCREEN_WIDTH // 2 - 25, 'y': 560, 'width': 50, 'height': 40, 'name': 'obs_w', 'gate_with' : 2, 'access' : 'left_right', 'orientation' : 'vertical'},  
        '4': {'x': 0, 'y': 300, 'width': 150, 'height': 50, 'name': 'obs_w', 'gate_with' : 5, 'access' : 'bottom_top', 'orientation' : 'horizontal'},
        '5': {'x': SCREEN_WIDTH // 4 + 50, 'y': 300, 'width': 125, 'height': 50, 'name': 'obs_w', 'gate_with' : 4, 'access' : 'bottom_top', 'orientation' : 'horizontal'}, 
        '6': {'x': SCREEN_WIDTH // 2 + 25, 'y': 380, 'width': 150, 'height': 50, 'name': 'obs_w', 'gate_with' : 7, 'access' : 'bottom_top', 'orientation' : 'horizontal'}, 
        '7': {'x': SCREEN_WIDTH - 105, 'y': 380, 'width': 110, 'height': 50, 'name': 'obs_w', 'gate_with' : 6, 'access' : 'bottom_top', 'orientation' : 'horizontal'}, 
    
}



# Create walls

wall = Walls(pygame)
wall.clear_walls()
walls = wall.make_wall(LEVEL_4_WALLS)



# Create the agent
agent = Agent("shanila", 15)
agent.agent_reset(SCREEN_WIDTH, SCREEN_HEIGHT)

# class Agent:
#     def __init__(self, agent_name, agent_index):
#          # positional attributes
#         self.previous_position = np.array([0, 0], dtype=np.float32)
#         self.current_position = None
#         self.same_position = False
#         self.current_step = 0
#         self.action = None
#         # these are for the angular motion of the agent
#         self.angle = 0
#         self.center = 0
#         self.direction = 0
#         self.direction_end = 0
#         self.radius = 15




def cast_and_update_rays(agent, walls):
    num_rays = 13
    start_angle = agent.angle - 65  # 65 degrees to the left
    end_angle = agent.angle + 65  # 65 degrees to the right
    angle_step = 10  # One ray every 10 degrees
    ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()
    ray_angles = [angle%360 for angle in ray_angles]

    
    
    updated_ray_lengths = []

    for angle in ray_angles:
        ray_direction = np.array([math.cos(math.radians(angle)), math.sin(math.radians(angle))])
        ray_direction /= np.linalg.norm(ray_direction)  # Normalize the direction vector
        x1, y1 = agent.center
        x2, y2 = agent.center + SCREEN_WIDTH * ray_direction  # Set an initial endpoint far away
        length = None
        for wall in walls:
            wall_vertices = [wall.topleft, wall.topright, wall.bottomright, wall.bottomleft]
            for i in range(len(wall_vertices)):
                x3, y3 = wall_vertices[i]
                x4, y4 = wall_vertices[(i + 1) % len(wall_vertices)]
            # x3, y3, x4, y4 = wall['x'], wall['y'], wall['x'] + wall['width'], wall['y'] + wall['height']

            denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if denominator == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator



            # den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            # if den == 0:
            #     continue

            # t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / den
            # u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / den

            epsilon = 1e-5  # Small epsilon value

            if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
                intersection_x = x1 + t * (x2 - x1)
                intersection_y = y1 + t * (y2 - y1)

                distance = math.hypot(intersection_x - x1, intersection_y - y1)
                if length is None or distance < length:
                    length = distance
            

            # if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
            #     intersection_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
            #     wall_length = np.linalg.norm(intersection_point - agent.current_position)
            #     if wall_length < length:
            #         length = wall_length  # Update length if this intersection point is closer

        # if length == float('inf'):
        #     # If no intersections were found, set the length to a finite value (e.g., distance to the screen boundary)
        #     length = SCREEN_WIDTH  # Adjust this value accordingly

        updated_ray_lengths.append(length)

    return ray_angles, updated_ray_lengths

# Example usage:

# updated_angles, updated_lengths = cast_and_update_rays(agent, walls)
# print(updated_angles, updated_lengths)





def render():
    screen.fill((230,229,239))
    # agent_pos = (int(agent.current_position[0]), int(agent.current_position[1]))
    pygame.draw.circle(screen, (255, 0, 0), agent.center, agent.radius)
    ray_angles, ray_lengths = cast_and_update_rays(agent, walls)


    # for angle, length in zip(ray_angles, ray_lengths):
    #     # if math.isfinite(length):  # Check if the length is finite (ray intersects with a wall)
    #     # end_point = (
    #     #     int(agent.current_position[0] + (length * math.cos(math.radians(angle)))),
    #     #     int(agent.current_position[1] + (length * math.sin(math.radians(angle))))
    #     # )
    #     # pygame.draw.line(screen, (0, 0, 255), agent.center, end_point, 2)
    #     print(angle, length)
        
    #     if length is not None:
    #         ray_end_x = agent.center[0] + length * math.cos(math.radians(angle))
    #         ray_end_y = agent.center[1] + length * math.sin(math.radians(angle))
    #         pygame.draw.line(screen, RED, agent.center, (ray_end_x, ray_end_y), 1)
    #     else:
    #         ray_end_x = agent.center[0] + 1000 * math.cos(math.radians(angle))
    #         ray_end_y = agent.center[1] + 1000 * math.sin(math.radians(angle))
    #         pygame.draw.line(screen, BLUE, agent.center, (ray_end_x, ray_end_y), 1)
        
    for wall in LEVEL_4_WALLS.values():
        pygame.draw.rect(screen, (120,130,164), (wall['x'], wall['y'], wall['width'], wall['height']))
    pygame.draw.rect(screen, (0, 0, 0), (650, 0, 150, 150), 2)
    window.blit(screen, screen.get_rect())
    pygame.display.update()
    clock.tick(60)


running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    key = pygame.key.get_pressed()
    if key[pygame.K_UP]:
        agent.step_update(2, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_LEFT]:
        agent.step_update(1, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_RIGHT]:
        agent.step_update(0, SCREEN_WIDTH, SCREEN_HEIGHT)

    render()

pygame.display.quit()
pygame.quit()


In [ ]:
angle = 0
start_angle = angle - 65  # 65 degrees to the left
end_angle = angle + 65  # 65 degrees to the right
angle_step = 10  # One ray every 10 degrees
ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()
ray_angles = [angle%360 for angle in ray_angles]
ray_angles

In [ ]:
pygame.quit()

In [ ]:
LEVEL_4_WALLS.items()

In [ ]:
pygame.init()
pygame.display.init()
window = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()
screen = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create walls

wall = Walls(pygame)
wall.clear_walls()
walls = wall.make_wall(LEVEL_4_WALLS)

walls[0].topleft

In [ ]:
import pygame
import sys
import math

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10
move_step = 1


# Define objects with arbitrary shapes
objects = {
    "special_wall": [(100, 100), (120, 100), (120, 200), (100, 200)],
    "wall2": [(200, 300), (220, 300), (220, 400), (200, 400)],
    "wall3": [(300, 200), (320, 200), (320, 300), (300, 300)],
    "circular_wall": [(400, 400), (430, 400), (430, 430), (400, 430)]
}

def is_ray_blocked(ray_start, ray_angle):
    closest_intersection = None
    
    x1, y1 = ray_start
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

    for object_name, object_vertices in objects.items():
        for i in range(len(object_vertices)):
            x3, y3 = object_vertices[i]
            x4, y4 = object_vertices[(i + 1) % len(object_vertices)]

            denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if denominator == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

            epsilon = 1e-5  # Small epsilon value

            if -epsilon <= t <= 1 + epsilon and 0 <= u <= 1:
                intersection_x = x1 + t * (x2 - x1)
                intersection_y = y1 + t * (y2 - y1)

                distance = math.hypot(intersection_x - x1, intersection_y - y1)
                if closest_intersection is None or distance < closest_intersection:
                    closest_intersection = distance

    return closest_intersection

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        predator_x -= move_step
    if keys[pygame.K_RIGHT]:
        predator_x += move_step
    if keys[pygame.K_UP]:
        predator_y -= move_step
    if keys[pygame.K_DOWN]:
        predator_y += move_step

    screen.fill(WHITE)

    for angle in range(360):
        ray_start = (predator_x, predator_y)
        ray_angle = angle
        closest_intersection = is_ray_blocked(ray_start, ray_angle)

        if closest_intersection is not None:
            ray_end_x = ray_start[0] + closest_intersection * math.cos(math.radians(ray_angle))
            ray_end_y = ray_start[1] + closest_intersection * math.sin(math.radians(ray_angle))
            pygame.draw.line(screen, RED, ray_start, (ray_end_x, ray_end_y), 1)
        else:
            ray_end_x = ray_start[0] + 1000 * math.cos(math.radians(ray_angle))
            ray_end_y = ray_start[1] + 1000 * math.sin(math.radians(ray_angle))
            pygame.draw.line(screen, BLUE, ray_start, (ray_end_x, ray_end_y), 1)

    pygame.draw.circle(screen, RED, (predator_x, predator_y), predator_radius)

    for object_name, object_vertices in objects.items():
        pygame.draw.polygon(screen, GREEN, object_vertices)
        

    pygame.display.flip()

pygame.quit()
sys.exit()


In [ ]:
pygame.quit()

In [82]:
import pygame
import sys
import math
import numpy as np
from Agents.agent import Agent

def rotate_agent(agent, angle_increment):
    agent.angle += angle_increment
    agent.angle = agent.angle % 360
    agent.get_direction()
    
# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator Agent Environment")

# Define colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Predator agent position and size
predator_x, predator_y = WIDTH // 2, HEIGHT // 2
predator_radius = 10
move_step = 1

# Create an instance of the Agent class
predator_agent = Agent(agent_name="Predator", agent_index=1)
predator_agent.current_position = np.array([predator_x, predator_y], dtype=np.float32)


# Define walls with new format
LEVEL_5_WALLS = {
    "1": {"x": 150, "y": 120, "width": 100, "height": 30},
    "2": {"x": 300, "y": 80,  "width": 30,  "height": 60},
    "3": {"x": 450, "y": 120, "width": 100, "height": 30},
    "4": {"x": 600, "y": 180, "width": 30,  "height": 100},
    "5": {"x": 500, "y": 400, "width": 100, "height": 30},
    "6": {"x": 150, "y": 250, "width": 30,  "height": 200},
    "7": {"x": 300, "y": 500, "width": 150, "height": 30}
}

def is_ray_blocked(ray_start, ray_angle):
    lengths = None

    x1, y1 = ray_start
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))

    for wall_name, wall_data in LEVEL_5_WALLS.items():
        x3, y3 = wall_data["x"], wall_data["y"]
        x4, y4 = x3 + wall_data["width"], y3 + wall_data["height"]

        for side in [(x3, y3, x4, y3), (x4, y3, x4, y4), (x4, y4, x3, y4), (x3, y4, x3, y3)]:
            x5, y5, x6, y6 = side

            denominator = (x1 - x2) * (y5 - y6) - (y1 - y2) * (x5 - x6)

            if denominator == 0:
                continue

            t = ((x1 - x5) * (y5 - y6) - (y1 - y5) * (x5 - x6)) / denominator
            u = -((x1 - x2) * (y1 - y5) - (y1 - y2) * (x1 - x5)) / denominator

            epsilon = 1e-5  # Small epsilon value

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersection_x = x1 + t * (x2 - x1)
                intersection_y = y1 + t * (y2 - y1)

                # Calculate the distance from the ray start to the intersection point
                distance = math.sqrt((intersection_x - x1) ** 2 + (intersection_y - y1) ** 2)

                if lengths is None or distance < lengths:
                    lengths = distance

    return lengths


# Inside your game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        rotate_agent(predator_agent, -10)  # Rotate counter-clockwise
    if keys[pygame.K_RIGHT]:
        rotate_agent(predator_agent, 10)   # Rotate clockwise
    if keys[pygame.K_UP]:
        predator_agent.step_update(2, 1, WIDTH, HEIGHT)  # Move forward
    if keys[pygame.K_DOWN]:
        predator_agent.step_update(3, 1, WIDTH, HEIGHT)  # Move backward

    screen.fill(WHITE)

    # Render walls
    for wall_name, wall_data in LEVEL_5_WALLS.items():
        pygame.draw.rect(screen, GREEN, (wall_data["x"], wall_data["y"], wall_data["width"], wall_data["height"]))

    # Render rays
    for angle in range(360):
        ray_start = predator_agent.current_position.astype(int)  # Convert to int
        ray_angle = angle
        lengths = is_ray_blocked(ray_start, ray_angle)

        if lengths is not None:
            color = (255, 0, 0)
            ray_end_x = ray_start[0] + lengths * math.cos(math.radians(ray_angle))
            ray_end_y = ray_start[1] + lengths * math.sin(math.radians(ray_angle))
            pygame.draw.line(screen, color, ray_start, (int(ray_end_x), int(ray_end_y)), 1)
        else:
            color = (0, 0, 255)
            ray_end_x = ray_start[0] + 1000 * math.cos(math.radians(ray_angle))
            ray_end_y = ray_start[1] + 1000 * math.sin(math.radians(ray_angle))
            pygame.draw.line(screen, color, ray_start, (int(ray_end_x), int(ray_end_y)), 1)

    # Render predator agent using agent's current position
    pygame.draw.circle(screen, RED, (int(predator_agent.current_position[0]), int(predator_agent.current_position[1])), predator_radius)

    pygame.display.flip()

pygame.quit()
sys.exit()



SystemExit: 

In [ ]:
pygame.quit()

final

In [35]:
import math
import pygame
import numpy as np
from Walls.wall_class import Walls
from Agents.agent import Agent
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], '..'))


# class Agent:
#     def __init__(self, agent_name, agent_index):
#         # positional attributes
#         self.previous_position = np.array([0, 0], dtype=np.float32)
#         self.current_position = np.array([0, 0], dtype=np.float32)
#         self.same_position = False

#         self.current_step = 0
#         self.action = None

#         # these are for the angular motion of the agent
#         self.angle = 0
#         self.center = np.array([WIDTH // 2, HEIGHT // 2], dtype=np.float32)
#         self.direction = 0
#         self.direction_end = 0
#         self.radius = 15

    # def rotate(self, angle):
    #     # Update the agent's angle
    #     self.angle += angle

    #     # Update the agent's current position based on the new angle
    #     self.current_position[0] = self.center[0] + self.radius * np.cos(np.radians(self.angle))
    #     self.current_position[1] = self.center[1] + self.radius * np.sin(np.radians(self.angle))


# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Ray Casting Example")

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Create an instance of the Agent class
predator_agent = Agent("Predator", 1)
predator_agent.agent_reset(WIDTH, HEIGHT)

# Define walls with new format
LEVEL_5_WALLS = {
    "1": {"x": 150, "y": 120, "width": 100, "height": 30},
    "2": {"x": 300, "y": 80, "width": 30, "height": 60},
    "3": {"x": 450, "y": 120, "width": 100, "height": 30},
    "4": {"x": 600, "y": 180, "width": 30, "height": 100},
    "5": {"x": 500, "y": 400, "width": 100, "height": 30},
    "6": {"x": 150, "y": 250, "width": 30, "height": 200},
    "7": {"x": 300, "y": 500, "width": 150, "height": 30}
}

wall_object = Walls(pygame)
walls = wall_object.make_wall(LEVEL_5_WALLS)

def get_cast_ray_angles(agent_angle):
    start_angle = agent_angle - 65  # 65 degrees to the left
    end_angle = agent_angle + 65  # 65 degrees to the right
    angle_step = 10  # One ray every 10 degrees
    ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()
    ray_angles = [angle%360 for angle in ray_angles]

    return ray_angles

def is_ray_blocked(agent, wall_list):
    
    ray_angles = get_cast_ray_angles(agent.angle)
    ray_lengths = []
    

    for ray_angle in ray_angles:
        x1, y1 = agent.center
        x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))
        lengths = None

        for wall in wall_list:
            x3, y3 = wall.x, wall.y
            x4, y4 = wall.topright[0], wall.bottomright[1]

            for side in [(x3, y3, x4, y3), (x4, y3, x4, y4), (x4, y4, x3, y4), (x3, y4, x3, y3)]:
                x5, y5, x6, y6 = side

                denominator = (x1 - x2) * (y5 - y6) - (y1 - y2) * (x5 - x6)

                if denominator == 0:
                    continue

                t = ((x1 - x5) * (y5 - y6) - (y1 - y5) * (x5 - x6)) / denominator
                u = -((x1 - x2) * (y1 - y5) - (y1 - y2) * (x1 - x5)) / denominator

                epsilon = 1e-5  # Small epsilon value

                if 0 <= t <= 1 and 0 <= u <= 1:
                    intersection_x = x1 + t * (x2 - x1)
                    intersection_y = y1 + t * (y2 - y1)

                    # Calculate the distance from the ray start to the intersection point
                    distance = math.sqrt((intersection_x - x1) ** 2 + (intersection_y - y1) ** 2)

                    if lengths is None or distance < lengths:
                        lengths = distance
              
        if lengths is None:
            lengths = 1000
            
        ray_lengths.append(lengths)
    return ray_lengths, ray_angles

# Pygame main loop
def main():
    clock = pygame.time.Clock()

    # Initial ray start position (center of the screen)
    # ray_start = [WIDTH // 2, HEIGHT // 2]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        keys = pygame.key.get_pressed()
        
        # Update agent's rotation based on arrow key inputs
        if keys[pygame.K_LEFT]:
            predator_agent.step_update(action=1, speed_factor=0.022, range_x=WIDTH, range_y=HEIGHT)
        if keys[pygame.K_RIGHT]:
            predator_agent.step_update(action=0, speed_factor=0.022, range_x=WIDTH, range_y=HEIGHT)
        if keys[pygame.K_UP]:
            predator_agent.step_update(2, 0.022, WIDTH, HEIGHT)

        # Clear the screen
        screen.fill(WHITE)

        # Draw the agent
        pygame.draw.circle(screen, RED, predator_agent.center, 10)

        lengths, angles = is_ray_blocked(predator_agent, walls)
        # print(len(lengths), len(angles))
        print(lengths)
        print(angles)
        for length, angle in zip(lengths, angles):
            
            if length < 1000:
                color = RED
            else:
                color = BLUE
                
        

            # Draw the ray with the calculated length
            ray_end_x = predator_agent.center[0] + length * np.cos(np.radians(angle))
            ray_end_y = predator_agent.center[1] + length * np.sin(np.radians(angle))
            start_pos = predator_agent.center
            end_pos = (int(ray_end_x), int(ray_end_y))
            pygame.draw.line(screen, color, start_pos, end_pos, 1)

        # Render the walls
        for wall_data in LEVEL_5_WALLS.values():
            pygame.draw.rect(screen, BLACK, (wall_data["x"], wall_data["y"], wall_data["width"], wall_data["height"]))

        pygame.display.flip()
        clock.tick(60)  # Limit the frame rate to 60 frames per second

if __name__ == "__main__":
    main()


[177.46511873643732, 130.75850967158237, 106.06601717798213, 1000, 579.7193878723624, 439.9923766742853, 225.85946344725318, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
[295, 305, 315, 325, 335, 345, 355, 5, 15, 25, 35, 45, 55, 65]
[177.46511873643732, 130.75850967158237, 106.06601717798213, 1000, 579.7193878723624, 439.9923766742853, 225.85946344725318, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
[295, 305, 315, 325, 335, 345, 355, 5, 15, 25, 35, 45, 55, 65]
[177.46511873643732, 130.75850967158237, 106.06601717798213, 1000, 579.7193878723624, 439.9923766742853, 225.85946344725318, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
[295, 305, 315, 325, 335, 345, 355, 5, 15, 25, 35, 45, 55, 65]
[177.46511873643732, 130.75850967158237, 106.06601717798213, 1000, 579.7193878723624, 439.9923766742853, 225.85946344725318, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
[295, 305, 315, 325, 335, 345, 355, 5, 15, 25, 35, 45, 55, 65]
[177.46511873643732, 130.75850967158237, 106.06601717798213, 1000, 579.719387872

SystemExit: 

In [106]:
pygame.quit()

In [9]:
import math
import pygame
import numpy as np
from Walls.wall_class import Walls
from Agents.agent import Agent
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], '..'))


# Set up display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Ray Casting Example")

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Create an instance of the Agent class
predator_agent = Agent("Predator", 1)
predator_agent.agent_reset(WIDTH, HEIGHT)

# Define walls with new format
LEVEL_5_WALLS = {
    "1": {"x": 150, "y": 120, "width": 100, "height": 30},
    "2": {"x": 300, "y": 80, "width": 30, "height": 60},
    "3": {"x": 450, "y": 120, "width": 100, "height": 30},
    "4": {"x": 600, "y": 180, "width": 30, "height": 100},
    "5": {"x": 500, "y": 400, "width": 100, "height": 30},
    "6": {"x": 150, "y": 250, "width": 30, "height": 200},
    "7": {"x": 300, "y": 500, "width": 150, "height": 30}
}

wall_object = Walls(pygame)
walls = wall_object.make_wall(LEVEL_5_WALLS)

def get_cast_ray_angles(agent_angle):
    start_angle = agent_angle - 65  # 65 degrees to the left
    end_angle = agent_angle + 65  # 65 degrees to the right
    angle_step = 10  # One ray every 10 degrees
    ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()
    ray_angles = [angle%360 for angle in ray_angles]

    return ray_angles

def is_ray_blocked(agent, ray_angle):
    
    # ray_lengths = []
    lengths = None
    # ray_angles = get_cast_ray_angles(agent.angle)

    x1, y1 = agent.center
    x2, y2 = x1 + 1000 * math.cos(math.radians(ray_angle)), y1 + 1000 * math.sin(math.radians(ray_angle))
        

    for wall_key, wall_value in LEVEL_5_WALLS.items():
        x3, y3 = wall_value["x"], wall_value["y"]
        x4, y4 = x3 + wall_value["width"], y3 + wall_value["height"]

        for side in [(x3, y3, x4, y3), (x4, y3, x4, y4), (x4, y4, x3, y4), (x3, y4, x3, y3)]:
            x5, y5, x6, y6 = side

            denominator = (x1 - x2) * (y5 - y6) - (y1 - y2) * (x5 - x6)

            if denominator == 0:
                continue

            t = ((x1 - x5) * (y5 - y6) - (y1 - y5) * (x5 - x6)) / denominator
            u = -((x1 - x2) * (y1 - y5) - (y1 - y2) * (x1 - x5)) / denominator

            epsilon = 1e-5  # Small epsilon value

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersection_x = x1 + t * (x2 - x1)
                intersection_y = y1 + t * (y2 - y1)

                # Calculate the distance from the ray start to the intersection point
                distance = math.sqrt((intersection_x - x1) ** 2 + (intersection_y - y1) ** 2)

                if lengths is None or distance < lengths:
                    lengths = distance

    # ray_lengths.append(lengths)
    return lengths
# Pygame main loop
def main():
    clock = pygame.time.Clock()

    # Initial ray start position (center of the screen)
    # ray_start = [WIDTH // 2, HEIGHT // 2]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        keys = pygame.key.get_pressed()
        
        # Update agent's rotation based on arrow key inputs
        if keys[pygame.K_LEFT]:
            predator_agent.step_update(action=1, speed_factor=0.022, range_x=WIDTH, range_y=HEIGHT)
        if keys[pygame.K_RIGHT]:
            predator_agent.step_update(action=0, speed_factor=0.022, range_x=WIDTH, range_y=HEIGHT)
        if keys[pygame.K_UP]:
            predator_agent.step_update(2, 0.022, WIDTH, HEIGHT)

        # Clear the screen
        screen.fill(WHITE)

        # Draw the agent
        pygame.draw.circle(screen, RED, predator_agent.center, 10)

        angles = get_cast_ray_angles(predator_agent.angle)
        for angle in angles:
            length = is_ray_blocked(predator_agent, angle)
            if length is not None:
                color = RED
            else:
                color = BLUE
                length = 1000  # Maximum length when there's no intersection

            # Draw the ray with the calculated length
            ray_end_x = predator_agent.center[0] + length * np.cos(np.radians(angle))
            ray_end_y = predator_agent.center[1] + length * np.sin(np.radians(angle))
            start_pos = predator_agent.center
            end_pos = (int(ray_end_x), int(ray_end_y))
            pygame.draw.line(screen, color, start_pos, end_pos, 1)

        # # Iterate through 360 degrees
        # for angle in angles:
        #     ray_angle = angle
        #     length = is_ray_blocked(predator_agent.current_position, ray_angle)

        #     if length is not None:
        #         color = RED
        #     else:
        #         color = BLUE
        #         length = 1000  # Maximum length when there's no intersection

        #     # Draw the ray with the calculated length
        #     ray_end_x = predator_agent.center[0] + length * np.cos(np.radians(ray_angle))
        #     ray_end_y = predator_agent.center[1] + length * np.sin(np.radians(ray_angle))
        #     start_pos = predator_agent.center
        #     end_pos = (int(ray_end_x), int(ray_end_y))
        #     pygame.draw.line(screen, color, start_pos, end_pos, 1)

        # Render the walls
        for wall_data in LEVEL_5_WALLS.values():
            pygame.draw.rect(screen, BLACK, (wall_data["x"], wall_data["y"], wall_data["width"], wall_data["height"]))

        pygame.display.flip()
        clock.tick(60)  # Limit the frame rate to 60 frames per second

if __name__ == "__main__":
    main()


SystemExit: 

c:\Users\nehli\anaconda3\envs\myenv1\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
